## Optimize the primer regions used

Code to find the subsequences of the given primers for which the greatest number of sequences can be used.

In [ ]:
# Modified versions of the functions used to extract the region between two primers, 
# by doing a pairwise alignment to a referense sequence for which the primer positions
# are known.

from Bio import SeqIO
from Bio.Align import PairwiseAligner
import regex as re
from itertools import product

def align_to_reference(query_seq, reference_seq):
    """
    Align the query sequence to the reference sequence and return alignment details.
    """
    aligner = PairwiseAligner()
    aligner.mode = 'global'
    aligner.match_score = 1
    aligner.mismatch_score = -1
    aligner.open_gap_score = -2
    aligner.extend_gap_score = -1

    alignment = aligner.align(reference_seq, query_seq)[0]  # Get the best alignment
    aligned_ref = alignment.aligned[0]
    aligned_query = alignment.aligned[1]
    
    return aligned_ref, aligned_query

def extract_region_of_interest(aligned_ref, aligned_query, ref_start, ref_end):
    """
    Extract the region of interest from the aligned query sequence based on reference coordinates.
    """
    query_start = None
    query_end = None

    # Map reference positions to query sequence positions
    for ref_range, query_range in zip(aligned_ref, aligned_query):
        # Check if the primer region falls within this reference range
        if ref_start >= ref_range[0] and ref_start < ref_range[1]:
            query_start = query_range[0] + (ref_start - ref_range[0])
        if ref_end > ref_range[0] and ref_end <= ref_range[1]:
            query_end = query_range[0] + (ref_end - ref_range[0])
    
    return query_start, query_end

def process_sequences(fasta_file, reference_seq, primer_fwd, primer_rev):
    """
    Extract sequences between primers based on alignment to a reference sequence.
    """
    # Find positions of primers in the reference sequence
    ref_start = reference_seq.index(primer_fwd) + len(primer_fwd)
    ref_end = reference_seq.index(primer_rev)

    not_found = []

    for record in SeqIO.parse(fasta_file, "fasta"):
        seq = str(record.seq).upper()

        # Align sequence to reference
        aligned_ref, aligned_query = align_to_reference(seq, reference_seq)

        start, end = extract_region_of_interest(seq, aligned_ref, aligned_query, ref_start, ref_end)

        if start is None or end is None:
            not_found.append(record.id)
    
    return not_found

def find_primers(sequence, primer, length):
    '''
    Find all possible subsequences of a given length from a primer, 
    that only exist once in the reference organism
    Input: Reference sequence, primer, length of subsequence primers
    Output: List of all possible subsequence primers that are found just once in the reference
    '''
    out_list = []
    for i in range(len(primer)-length):
        primer_subseq = primer[i:i+length]
        matches = re.findall(primer_subseq, sequence)
        if len(matches) == 1:
            out_list.append(primer_subseq)
    
    return out_list

In [2]:
# Define the reference sequence and primers
for record in SeqIO.parse("Ecoli.fasta", "fasta"):
    ecoli = str(record.seq)

# Primers from extract_sequences_16s
v1_full = 'GTAACAGGAAGAAGCTTGCTTCTTTGCTGAC'
v9_full = 'GTAGGTAGCTTAACCTTCGGGAGGGCGCTTA'

v3_full = 'GTACTTTCAGCGGGGAGGAAGGGAGTAAAGTTAATACCTTTGCTCATTGACGTTACCCGCAGAAG'
v4_full = 'CGCACGCAGGCGGUUUGUUAAGUCAGAUGUGAAAUCCCCGGGCUCAACCUGGGAACUGCAUCUGAUACUGGCAAGCUUGAGUCUCGUAGAGGGGGGUAGAAUUCCAG'

In [ ]:
# Find all possible subsequence primers of lengths 4, 5, and 6
# v1
v1_four = find_primers(ecoli, v1_full, 4)
v1_five = find_primers(ecoli, v1_full, 5)
v1_six = find_primers(ecoli, v1_full, 6)
v1_all = v1_four + v1_five + v1_six

# v9
v9_four = find_primers(ecoli, v9_full, 4)
v9_five = find_primers(ecoli, v9_full, 5)
v9_six = find_primers(ecoli, v9_full, 6)
v9_all = v9_four + v9_five + v9_six

# v3
v3_four = find_primers(ecoli, v3_full[:round(len(v3_full)/2)], 4)
v3_five = find_primers(ecoli, v3_full[:round(len(v3_full)/2)], 5)
v3_six = find_primers(ecoli, v3_full[:round(len(v3_full)/2)], 6)
v3_all = v3_four + v3_five + v3_six

# v4
v4_four = find_primers(ecoli, v4_full[round(len(v4_full)/2):], 4)
v4_five = find_primers(ecoli, v4_full[round(len(v4_full)/2):], 5)
v4_six = find_primers(ecoli, v4_full[round(len(v4_full)/2):], 6)
v4_all = v4_four + v4_five + v4_six

In [ ]:
# Loop through all possible combinations for the v1-v9 region
# Save the primer combination and the number of sequences in which the
# region of interest cannot be found using that specific primer pair

primers_errors_v1v9 = {}

for v1, v9 in list(product(v1_all, v9_all)):
    errors = process_sequences("Vaginal_species_all.fasta", ecoli, v1, v9)
    primers_errors_v1v9[(v1, v9)] = len(errors)

In [ ]:
# Same but for the v3-v4 region

primers_errors_v3v4 = {}

for v3, v4 in list(product(v3_all, v4_all)):
    errors = process_sequences("Vaginal_species_all.fasta", ecoli, v3, v4)
    primers_errors_v3v4[(v3, v4)] = len(errors)

In [12]:
# Write the results to files
import csv

with open('optimized_v1v9.csv', "w") as out_file:
    writer = csv.writer(out_file)
    for key, value in primers_errors_v1v9.items():
       writer.writerow([key, value])

with open('optimized_v3v4.csv', "w") as out_file:
    writer = csv.writer(out_file)
    for key, value in primers_errors_v3v4.items():
       writer.writerow([key, value])